PREDICTION SCRIPT

The best algorithm for prediction has been found to be RandomTreeRegressor. We are going to fit it to the dataset and then just predict the values and save them.

Importing dataset and test set

In [1]:
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = pd.read_csv('../dataset_new.csv')
test_df = pd.read_csv('../dataset_test_new.csv')
target = df.iloc[:, 31]
target_test = test_df.iloc[:, 31]
columns_to_drop = df.iloc[:, [0, 1, 2, 12, 13, 14, 20, 21, 22, 23, 28, 31, 32]].columns
df_dropped = df.drop(columns=columns_to_drop)
df_test_dropped = test_df.drop(columns=columns_to_drop)

In [10]:
def clean_and_convert(column):
    column = column.str.replace(',', '.', regex=False)
    column = column.str.replace('+', '', regex=False)
    return pd.to_numeric(column, errors='coerce')

def data_nan(data):
    data.replace('N.A', np.nan, inplace=True)
    data['línea 3'] = data['línea 3'].astype(float)
    data['línea 3.1'] = data['línea 3.1'].astype(float)
    data['Viabilidad final cultivo_cultivo_final'] = pd.to_numeric(data['Viabilidad final cultivo_cultivo_final'], errors='coerce')
    data['Viabilidad final cultivo_cultivo_final'] = data['Viabilidad final cultivo_cultivo_final'].astype(float)
    for column in data.select_dtypes(include=[np.number]).columns:
        mean_value = data[column].mean()
        data[column].fillna(mean_value, inplace=True)
    for col in data.select_dtypes(include=['object']).columns:
        data[col] = clean_and_convert(data[col])
    return data


In [11]:
data = data_nan(df_dropped)
data_test = data_nan(df_test_dropped)
# Definir características (X) y objetivo (y)
X = data
X_test_final = data_test
y = target

In [12]:
model = RandomForestRegressor(max_depth= 10, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split= 5, n_estimators= 200)

In [13]:
model.fit(X, y)

RandomForestRegressor(max_depth=10, max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=5, n_estimators=200)

In [14]:
y_final_pred = model.predict(X_test_final)

In [17]:
data = pd.DataFrame({
    'LOTE': test_df['LOTE'].astype(int),  # Ensure LOTE is of integer type
    'PRODUCTO 1': y_final_pred  # Your predictions
})
data.sort_values(by='LOTE', ascending=True, inplace=True)

# Format the PRODUCTO 1 values to ensure correct decimal representation
data['PRODUCTO 1'] = data['PRODUCTO 1'].apply(lambda x: f"{x:.2f}".replace(',', '.'))

# Save the DataFrame to a text file with '|' as the delimiter
data.to_csv('../Universidad Carlos III de Madrid (UC3M)_BioData_new/universityhack.predict', sep='|', header=False, index=False)